In [5]:
import pathlib
import pandas as pd

# import sys
# sys.path.append("/home/roshankern/Desktop/Github/IDR_stream/IDR_stream")
from IDR_stream.download import AsperaDownloader

In [6]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
# remove A1 wells because of irregular illumination
data_to_process = data_to_process[data_to_process["Well"] != "A1"]
data_to_process = data_to_process.reset_index(drop=True)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target
0,LT0001_02,A4,50,4,positive control,ENSG00000149503
1,LT0001_02,A15,50,15,negative control,negative control
2,LT0001_02,B2,50,26,negative control,negative control
3,LT0001_02,C1,50,49,positive control,ENSG00000149503
4,LT0001_02,C4,50,52,positive control,ENSG00000149503
...,...,...,...,...,...,...
6759,LT0603_06,O13,50,349,positive control,KIF11
6760,LT0603_06,O16,50,352,negative control,negative control
6761,LT0603_06,P3,50,363,negative control,negative control
6762,LT0603_06,P21,50,381,positive control,COPB


In [7]:
aspera_path = pathlib.Path("/home/roshankern/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
idr_id = "idr0013"
save_dir = pathlib.Path("data/")

downloader = AsperaDownloader(aspera_path, aspera_key_path, screens_path, idr_id, save_dir)
#downloader.download_image("LT0001_02", 4)